Data Project

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, expr
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, LongType
from pyspark.sql.avro.functions import from_avro

Create spark session

In [3]:
spark = SparkSession.builder \
    .appName("BinanceDataCleaning") \
    .config("spark.jars", "/home/jovyan/work/postgresql-42.7.5.jar") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

In [4]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "binance_trades") \
    .option("startingOffsets", "latest") \
    .load()

In [6]:
# Schema dữ liệu
schema = StructType([
    StructField("trade_id", LongType(), True),
    StructField("symbol", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("quantity", FloatType(), True),
    StructField("time", LongType(), True),
    StructField("is_buyer_maker", BooleanType(), True)
])

# Chuyển đổi dữ liệu JSON
df_parsed = df.selectExpr("CAST(value AS STRING)").select(from_json(col("value"), schema).alias("data")).select("data.*")

# Định dạng cột timestamp
df_transformed = df_parsed.withColumn("time", expr("cast(time/1000 as timestamp)"))


In [7]:
def write_to_postgres(df, epoch_id):
    if df.count() == 0:
        return  # 🔥 Tránh lỗi batch trống
    df.write \
        .format("jdbc") \
        .option("url", "jdbc:postgresql://postgres:5432/crypto_db") \
        .option("dbtable", "staging_transactions") \
        .option("user", "postgres") \
        .option("password", "password") \
        .option("driver", "org.postgresql.Driver") \
        .mode("append") \
        .save()

df_transformed.writeStream \
    .foreachBatch(write_to_postgres) \
    .outputMode("append") \
    .start() \
    .awaitTermination()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [12]:
staging_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/crypto_db") \
    .option("dbtable", "staging_transactions") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .load()

fact_df = staging_df.withColumn("total_value", col("price") * col("quantity"))

fact_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres:5432/crypto_db") \
    .option("dbtable", "fact_trades") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()


In [9]:
pg_url = "jdbc:postgresql://postgres:5432/crypto_db"
pg_properties = {
    "user": "postgres",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

In [13]:
df = spark.read.jdbc(url=pg_url, table="fact_trades", properties=pg_properties)


In [14]:
df.show(5)


+----------+-------+-------------+--------------------+------------------+--------------------+--------------+
|  trade_id| symbol|        price|            quantity|       total_value|                time|is_buyer_maker|
+----------+-------+-------------+--------------------+------------------+--------------------+--------------+
|4543354184|BTCUSDT|97059.9765625|6.399999838322401E-4|  62.1183834307576|2025-02-10 16:19:...|          true|
|4543362489|BTCUSDT|  97048.40625|5.999999848427251E-5| 5.822904227901063|2025-02-10 16:22:...|         false|
|4543362490|BTCUSDT|  97048.40625|5.999999848427251E-5| 5.822904227901063|2025-02-10 16:22:...|         false|
|4543362491|BTCUSDT|  97048.40625|5.999999848427251E-5| 5.822904227901063|2025-02-10 16:22:...|         false|
|4543362492|BTCUSDT|  97048.40625|2.999999924213625...|2.9114521139505314|2025-02-10 16:22:...|         false|
+----------+-------+-------------+--------------------+------------------+--------------------+--------------+
o

In [11]:
# Loại bỏ NULL
df = df.na.drop()

# Loại bỏ dữ liệu trùng lặp
df = df.dropDuplicates(["trade_id"])

# Chuyển đổi kiểu dữ liệu
df = df.withColumn("price", col("price").cast("double"))
df = df.withColumn("quantity", col("quantity").cast("double"))
df = df.withColumn("time", col("time").cast("timestamp"))

# Tạo cột tổng giá trị giao dịch
df = df.withColumn("total_value", expr("price * quantity"))

In [12]:
df.show(5)


+----------+-------+--------+--------+--------------------+--------------+-----------+
|  trade_id| symbol|   price|quantity|                time|is_buyer_maker|total_value|
+----------+-------+--------+--------+--------------------+--------------+-----------+
|4532361114|BTCUSDT|97529.57|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517742|
|4532361115|BTCUSDT|97529.57|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517742|
|4532361116|BTCUSDT|97529.38|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517628|
|4532361117|BTCUSDT|97529.38|  6.0E-5|2025-02-07 05:07:...|          true|  5.8517628|
|4532361118|BTCUSDT|97529.37|  1.2E-4|2025-02-07 05:07:...|          true| 11.7035244|
+----------+-------+--------+--------+--------------------+--------------+-----------+
only showing top 5 rows



In [13]:
df.write.jdbc(url=pg_url, table="staging_transactions", mode="overwrite", properties=pg_properties)

print("Dữ liệu đã làm sạch và lưu vào Staging Layer trong Data Warehouse!")


Dữ liệu đã làm sạch và lưu vào Staging Layer trong Data Warehouse!


In [3]:
# Đọc dữ liệu từ Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9094") \
    .option("subscribe", "binance_trades") \
    .option("startingOffsets", "latest") \
    .load()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.